In [1]:
import os 
import json 
import time
import random 
import datetime
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC   
from webdriver_manager.chrome import ChromeDriverManager


# function to take care of downloading file
def enable_download_headless(browser, download_dir):
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    params = {'cmd':'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': download_dir}}
    browser.execute("send_command", params)


def setup_driver(proxy=None, headless=True):
    # proxy: https://x.x.x.x:port
    options = webdriver.ChromeOptions()
    # options.binary_location = "/usr/local/bin/chromedriver"
    # options.headless = headless
    #options.add_argument("--window-size=1920,1200")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument("start-maximized")
    options.add_argument("--disable-blink-features") 
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-gpu")
    # options.add_argument("--remote-debugging-port=9222")
    options.add_argument("disable-infobars")
    # options.add_experimental_option("prefs", {
    #     "download.default_directory": "/home/alice/data",
    #     "download.prompt_for_download": False,
    #     "download.directory_upgrade": True,
    #     "safebrowsing_for_trusted_sources_enabled": False,
    #     "safebrowsing.enabled": False
    # })

    if proxy is None:
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        # return webdriver.Firefox()
    else:
        prox = Proxy()
        prox.proxy_type = ProxyType.MANUAL
        prox.http_proxy = proxy.split("//")[-1]
        prox.ssl_proxy = proxy.split("//")[-1]
        capabilities = webdriver.DesiredCapabilities.CHROME
        prox.add_to_capabilities(capabilities)
        return webdriver.Chrome(desired_capabilities=capabilities, options=options)


In [16]:



#url = "https://www.aphanti.com/blog/27/"
#url = "https://www.happycow.net/north_america/usa/"
# url = "https://www.redfin.com/county/1347/MA/Worcester-County"
# url = 'https://www.nasdaq.com/market-activity/stocks/screener'
url = 'https://www.nasdaq.com/market-activity/stocks/aapl/option-chain'

# proxy = random.choice(json.load(open('../src/pre-defined/proxies.json', 'r')))['proxy']
proxy = None 
# header = random.choice(json.load(open('../data/headers.json', 'r')))
print('proxy:', proxy)
# print('header:', header)

driver = setup_driver(proxy, headless=False)
enable_download_headless(driver, "/home/alice/data")

driver.get(url)
time.sleep(1)
driver.execute_script("window.scrollTo(0, 300)")
# WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "nasdaq-screener__download"))) #.click()
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "jupiter22-option-chain-filter-moneyness"))) #.click()
div = driver.find_element(By.CLASS_NAME, "jupiter22-option-chain-filter-moneyness")
print(div.text)


# print('navigator.webdriver: ', driver.execute_script("return navigator.webdriver;"))
# WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[name^='a-'][src^='https://www.google.com/recaptcha/api2/anchor?']"))) 
# WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[@id='recaptcha-anchor']"))).click() 

#open('test2.html', 'w').write(driver.page_source)

proxy: None
Moneyness
Near the Money


In [10]:
div = driver.find_element(By.CLASS_NAME, "nasdaq-screener__form-button--download")
print(div.text)
time.sleep(3)
driver.execute_script("arguments[0].click();", div)


Download CSV


In [17]:
driver.execute_script("window.scrollTo(0, 600)")


In [20]:
label = div.find_element(By.CLASS_NAME, "jupiter22-option-chain-filter-toggle-moneyness")
print(label.text)
label.click()

Near the Money


In [23]:
target = div.find_element("xpath", "//div[text()='All (Moneyness)']")
print(target.text)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[text()='All (Moneyness)']"}
  (Session info: chrome=127.0.6533.72); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5cc58c4de6ba <unknown>
#1 0x5cc58c1ae730 <unknown>
#2 0x5cc58c1fdd9b <unknown>
#3 0x5cc58c1fe081 <unknown>
#4 0x5cc58c1f2826 <unknown>
#5 0x5cc58c220a7d <unknown>
#6 0x5cc58c1f2719 <unknown>
#7 0x5cc58c220c1e <unknown>
#8 0x5cc58c23f1ca <unknown>
#9 0x5cc58c2207f3 <unknown>
#10 0x5cc58c1f0ec9 <unknown>
#11 0x5cc58c1f191e <unknown>
#12 0x5cc58c4a49eb <unknown>
#13 0x5cc58c4a8972 <unknown>
#14 0x5cc58c491e15 <unknown>
#15 0x5cc58c4a9502 <unknown>
#16 0x5cc58c476d2f <unknown>
#17 0x5cc58c4cd578 <unknown>
#18 0x5cc58c4cd750 <unknown>
#19 0x5cc58c4dd48c <unknown>
#20 0x7b3beb694ac3 <unknown>
